# 1고지

In [1]:
import numpy as np

In [2]:
class Variable: # 객체 선언
    """
    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.

    backward_backup
        재귀 방식의 역전파 코드
    backward
        반복문을 이용한 역전파 코드
    """
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray) or isinstance(data, np.float64): # 타입 비교
                raise TypeError(f'{type(data)} is not supported.') # Error 표시 및 코드 종료

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func): # self 객체에 사용되는 함수 객체를 설정하는 메서드
        self.creator = func 

    ## 1. 재귀 방식의 역전파
    def backward_backup(self):
        """
        Function Explanation
            if문 내에서 값의 역전파를 수행한 뒤, 결과값(x.grad)을 다시 x.backward() 재귀 함수 안의 backward의 파라미터값으로 넣어 최종 값을 뽑아냄

        Parameter
        ---------
        input
            Function Class에서 사용되는 멤버변수 값으로 연산에 사용되기 위해 들어오는 Variable 객체
        """
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward_backup() # 뒤의 인자를 받아 backward 수행 및 새로나온 결과를 다시 뒤의 인자로 활용해 재귀하는 형태

    ## 2. 반복문을 이용한 역전파
    def backward(self):
        """
        Function Explanation
            while문을 통해 반복문이 다시 들어가기 전 Variable의 creator 값을 존재유무에 따라 역전파의 추가 수행여부를 판단함
        """
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs: # 루프를 통한 역전파 수행
            f = funcs.pop() # 메모리 문제를 해결하기 위한 pop
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None: # 최초 노드로 진입할 경우 creator가 존재하지 않으므로 더이상의 수행이 진행되지 않음
                funcs.append(x.creator)

In [3]:
help(Variable)

Help on class Variable in module __main__:

class Variable(builtins.object)
 |  Variable(data)
 |  
 |  Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.
 |  
 |  Parameter
 |  ---------
 |  data : np.array
 |      실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
 |  grad : int, float
 |      역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
 |  creator : callable function
 |      Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨
 |  
 |  Functions
 |  ---------
 |  # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
 |  # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.
 |  
 |  backward_backup
 |      재귀 방식의 역전파 코드
 |  backward
 |      반복문을 이용한 역전파 코드
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  backward(self)
 |      Function Explanation
 |          while문을 통해 반복문이 다시 들어가기 전 Variable의 creator 값을 존재유무에 따라 역전파의 추가

In [5]:
print(Variable.__doc__)


    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.

    backward_backup
        재귀 방식의 역전파 코드
    backward
        반복문을 이용한 역전파 코드
    


In [6]:
class Function:
    """
    Class Function : Variable 객체가 수행하는 함수들을 가지고 있는 클래스

    Parameter
    ---------
    input : Variable (Class instance)
    output : Variable (Class instance)
        input.data에 들어오는 ndarray 값을 함수 클래스를 통해 순전파된 결과인 y을 가지고 as_array 함수로 ndarray로 형변환하고 Variable의 __init__ 메서드로 값을 넣어 생성된 Variable 객체

    Functions
    ---------
    __call__
        위 클래스는 function 객체를 다루고 function 객체 내부에 input을 함수를 사용하듯이 값을 받음을 알 수 있음
        return : output
    forward, backward
        순전파 및 역전파 수행하는 클래스로써 기반클래스의 기본 포함 메서드. 파생 클래스에서 위 메서드들을 생성하지 않고 호출하게 되면 NotImplementedError를 raise하게 됨
        NotImplementedError : 파생 클래스에서 구현되지 않았음을 알리는 에러
    """
    def __call__(self, input): # 객체를 함수처럼 호출하는 함수 객체로써 역할을 수행하기 위해 사용되는 매직 메서드
        x = input.data
        y = self.forward(x) # input이라는 variable 객체를 받아 순전파 수행 및 결과 저장

        output = Variable(as_array(y)) # 값이 ndarray 타입이 아닐 경우 변환해주고 variable로 감싸주는 형태
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [7]:
def as_array(x):
    """ 타입체크 및 ndarray 형태로 변환해주는 함수 """
    if np.isscalar(x):
        return np.array(x)
    return x

In [8]:
class Square(Function):
    """ Function이라는 기반 클래스로부터 상속받는 Square 파생 클래스 """
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [9]:
class Exp(Function):
    """ Function이라는 기반 클래스로부터 상속받는 Exp 파생 클래스 """
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gy = np.exp(x) * gy
        return gy

In [10]:
def numerical_diff(f, x, eps=1e-4):
    """ 수치 미분을 수행하는 전역 함수 """
    if type(x.data - eps) is np.float64:
        temp1 = np.array(x.data - eps)
        x0 = Variable(temp1)
    else:
        x0 = Variable(x.data - eps)
    if type(x.data + eps) is np.float64:
        temp2 = np.array(x.data + eps)
        x1 = Variable(temp2)
    else:
        x1 = Variable(x.data + eps)
    
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [11]:
def f(x): # 함수 테스트
    A = Square() # __call__ 메서드를 통해 객체를 그대로 생성
    B = Exp() # <__main__.Exp at 0x16a0a3257c0>
    C = Square() # <__main__.Square at 0x16a0afa3b80>
    return C(B(A(x)))

In [12]:
x = Variable(np.array(10.0))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100.0


In [13]:
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [14]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 검증
# assert 뒤의 값이 True가 아닐 경우, AssertionError 발생
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [15]:
# 역전파 1 (내부의 멤버변수를 다루지 않은 상태에서 backprop 수행)
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [16]:
# 역전파 2 (내부의 멤버변수를 차용해 backprop 수행)
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

In [17]:
print(x.grad)

3.297442541400256


In [18]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [19]:
# 함수 객체를 활용해 실제 수학적 연산을 수행하듯이 함수를 정의
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

def exp(x):
    f = Exp()
    return f(x)
    # return Exp()(x)

In [20]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # Sqaure, Exp, Sqaure 3개의 함수 객체를 혼용하여 순전파 수행 (e**(x**2))**2 형태가 됨
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [21]:
x = Variable(np.array(0.5))
x = Variable(None)

x = Variable(1.0) # float에 대해서 처리가 되어있지 않으므로 오류 발생 (np.float의 경우 처리됨)

TypeError: <class 'float'> is not supported.

In [26]:
import unittest

class SquareTest(unittest.TestCase):
    """
    unittest 모듈을 기본적으로 사용하며 테스트 주도 개발을 위한 용도로 release 이전 메서드들의 검증을 위해 사용
    Class SquareTest : unittest.TestCase 클래스를 상속받는 파생 클래스
    
    Functions
    ---------
    assertEqual
        들어오는 인자값을 비교해 같은지 여부를 bool 형식으로 반환
    assertTrue
        들어오는 bool값을 인자로 넣었을 때 True인지 확인하는 메서드

    using method
    ------------
    np.allclose : a, b, rtol, atol / <numpy module : https://numpy.org/doc/stable/reference/generated/numpy.allclose.html>
        a, b 인자값의 오차를 측정하는 함수로 |a - b| <= (atol + rtol*|b|) 라는 수식을 활용함
        큰 값으로 들어올 경우에 오차를 조금 유연하게 주기 위해서 단순하게 비교하지 않고 rtol*|b|라는 항을 사용한 것으로 추측
        return : bool
    """

    # Case 1
    def test_forward(self): # unittest를 상속받는 클래스는 필수적으로 def에 들어가는 함수 이름에 test를 넣어주어야 한다.
        x = Variable(np.array(2.))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected) # assertEqual 함수를 unittest.TestCase로부터 받아서 들어오는 인자값끼리 값을 비교한다.
    
    # Case 2 (include Case 1)
    def test_backward(self):
        x = Variable(np.array(3.))
        y = square(x)
        y.backward()
        expected = np.array(6.)
        self.assertEqual(x.grad, expected)

    # Case 3 (include Case 1, 2)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 주입
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

In [25]:
#########################################################
# test_forward(self) 함수만 선언하고 사용했을 때 결과 확인 #
#########################################################

# Case 1
unittest.main(argv=[''], verbosity=2, exit=False)

# .py 파일내에서 unittest.TestCase를 상속받는 클래스를 만든 경우에는
# 테스트 시 (첫 번째 경우)
# python -m unittest 파일명.py
# 라는 명령어를 터미널에 주어야 아래와 같은 실행 결과를 확인할 수 있음

# 또 다른 테스트 시 (두 번째 경우)
# 함수 본문 마지막에 unittest.main()을 추가한 뒤
# 정상적으로 python 파일명.py
# 를 터미널에서 실행해도 됨

test_forward (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [27]:
# Case 2
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [23]:
# Case 3
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok
test_gradient_check (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.012s

OK


### 터미널에서 .py에 있는 tests 디렉토리 내에 있는 테스트 파일들을 한꺼번에 검사하기 위해서는
#### python -m unittest discover tests
#### 라는 명령어를 주게 되면 test 폴더 내부에 있는 모든 test 파일들에 대해 검사를 수행한다.
#### 여기서 discover라는 하위 명령은 discover 뒤에 지정된 디렉터리에 대해 파일이 있는지 탐색하는 명령어.

# 2고지

In [28]:
class Function:
    """ Update Version (Level 2).
    Class Function : Variable 객체가 수행하는 함수들을 가지고 있는 클래스

    Parameter
    ---------
    input : Variable (Class instance)
    output : Variable (Class instance)
        input.data에 들어오는 ndarray 값을 함수 클래스를 통해 순전파된 결과인 y을 가지고 as_array 함수로 ndarray로 형변환하고 Variable의 __init__ 메서드로 값을 넣어 생성된 Variable 객체

    Functions
    ---------
    __call__
        위 클래스는 function 객체를 다루고 function 객체 내부에 input을 함수를 사용하듯이 값을 받음을 알 수 있음
        >> Update
            inputs 파라미터를 리스트 데이터로 받아서 처리. 각각의 값은 Variable 객체로 구성되어 있음
            list comprehension을 사용하여 묶여진 객체들을 다시 인덱싱하여 결론을 도출함
        return : outputs
        >> Update
            Variable을 담고있는 list container로써 값을 리턴함
    forward, backward
        순전파 및 역전파 수행하는 클래스로써 기반클래스의 기본 포함 메서드. 파생 클래스에서 위 메서드들을 생성하지 않고 호출하게 되면 NotImplementedError를 raise하게 됨
        NotImplementedError : 파생 클래스에서 구현되지 않았음을 알리는 에러
    """
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)

        outputs = [Variable(as_array(y)) for y in ys] # 순전파된 결과를 array로 변환한 다음 Variable의 인자값으로 주어 리스트에 다시 저장
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, x):
        raise NotImplementedError() # 파생 클래스에서 구현되지 않았음을 알리는 에러

    def backward(self, gy):
        raise NotImplementedError()

In [29]:
class Add(Function):

    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

    def backward(self, gy):
        return gy, gy

In [30]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [31]:
class Function:
    """ Update Version (Level 2-2).
    Class Function : Variable 객체가 수행하는 함수들을 가지고 있는 클래스

    Parameter
    ---------
    input : Variable (Class instance)
    output : Variable (Class instance)
        input.data에 들어오는 ndarray 값을 함수 클래스를 통해 순전파된 결과인 y을 가지고 as_array 함수로 ndarray로 형변환하고 Variable의 __init__ 메서드로 값을 넣어 생성된 Variable 객체

    Functions
    ---------
    __call__
        위 클래스는 function 객체를 다루고 function 객체 내부에 input을 함수를 사용하듯이 값을 받음을 알 수 있음        
        inputs 파라미터를 리스트 데이터로 받아서 처리. 각각의 값은 Variable 객체로 구성되어 있음
        list comprehension을 사용하여 묶여진 객체들을 다시 인덱싱하여 결론을 도출함
        >> Update
            Level 2 버전의 Function 클래스와 큰 차이는 없으나 처음에 받는 인자를 가변인자를 받을 수 있도록 설정되어 있음
            generation 설정 : 이전까지의 Function 클래스의 경우 단일 노드들간의 연산이었으나, 지금부터는 여러 노드들에 대한 연산(ex. 더하기, 빼기 등)도 수행해야 하기 때문에 여러 노드들의 역전파를 효율적으로 수행하기 위해 함수 단계마다 세대(generation)를 설정하여 우선순위를 매긴 다음 역전파를 수행하도록 함
        return : outputs
        >> Update
            inputs 값을 가변인자로 받기 때문에 1개인 경우에 대한 예외처리가 필요하여 삼항 연산자를 활용해 조건에 따라 return되는 값을 다르게 하였음
    forward, backward
        순전파 및 역전파 수행하는 클래스로써 기반클래스의 기본 포함 메서드. 파생 클래스에서 위 메서드들을 생성하지 않고 호출하게 되면 NotImplementedError를 raise하게 됨
        NotImplementedError : 파생 클래스에서 구현되지 않았음을 알리는 에러
    """
    def __call__(self, *inputs): # 제 2고지와 첫 Function과 다르게 inputs 값을 가변인자 형태로 받음
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # inputs는 Variable 객체의 리스트이므로 리스트를 분리해서 data 추출
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # 순전파된 결과를 array로 변환한 다음 Variable의 인자값으로 주어 리스트에 다시 저장
        
        self.generation = max([x.generation for x in inputs]) # 들어오는 값들의 generation값을 체크한 뒤 가장 높을 값을 현재 generation으로 설정
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError() # 파생 클래스에서 구현되지 않았음을 알리는 에러

    def backward(self, gy):
        raise NotImplementedError()

In [32]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y # 기반 클래스에서 isinstance 여부를 확인하여 tuple로 변환 처리됨

    def backward(self, gy):
        return gy, gy

In [33]:
def add(x0, x1): # 함수 객체 호출을 간편하게 하기 위해서 add 함수 정의
    return Add()(x0, x1)

In [34]:
class Variable:
    """ Update Version (Level 2).
    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨
    >> Update
        generation : Variable 객체가 어느 세대(generation)에 포함되어 있는지를 표현하기 위한 변수

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.
    >> Update
        __init__
            isinstance : 첫 번째 파라미터값의 타입과 두 번째 파라미터값의 동등 여부를 확인하는 함수
        cleargrad
            메모리 절약을 위해서 한 번 할당했던 변수를 또 다시 사용해야 할 경우 기존 메모리에 있던 Variable.grad 정보가 남아있기 때문에 할당을 해제해주어야 제대로 동작을 수행하게 된다.
        backward
            반복문을 이용한 역전파 코드
            while loop 내에서 zip 함수를 활용하여 x.grad에 중복되게 값이 들어갈 경우 기존 노드에 더해지도록 만들어 x + x = 2x 라는 식을 예로 들었을 때, gradient 값이 2로 정상 출력되게 하였음
    """
    def __init__(self, data):
        if data is not None:
            if isinstance(data, list):
                pass
            elif not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')
            

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self): # 한 번 사용했던 변수를 재사용하는 경우에 grad 값이 이미 존재하는 경우 충돌이 발생하므로 이를 clear해주는 메서드를 정의
        self.grad = None

    def set_creator(self, func): 
        self.creator = func
        # generation 추가. Node가 같은 층에서 여러 개가 들어갈 경우 역전파 수행 시 직렬로 수행되므로 한 층에 있는 다른 노드에 대해서 처리가 병렬적으로 되지 않기 때문에 우선순위를 매기는 generation 변수를 설정함
        self.generation = func.generation + 1

    def backward(self): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) # 가변인자로 값을 받아 역전파를 수행하고
            if not isinstance(gxs, tuple): # 타입을 체크한 뒤
                gxs = (gxs,)
            for x, gx in zip(f.inputs, gxs): # zip 함수를 활용해 값을 체크하되
                if x.grad is None:
                    x.grad = gx
                else: # x + x = 2x와 같이 x값을 두 번 사용하는 경우 값이 이미 연산되어 들어가므로 이에 대해서 예외 처리를 진행 & += 연산자를 사용하게 될 경우 메모리 주소를 공유하게 되므로 메모리 주소를 다르게 사용하기 위해서 x.grad = x.grad + gx 구문을 사용함
                    x.grad = x.grad + gx
                if x.creator is not None:
                    funcs.append(x.creator)

In [35]:
print(Variable.__doc__)

 Update Version (Level 2).
    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨
    >> Update
        generation : Variable 객체가 어느 세대(generation)에 포함되어 있는지를 표현하기 위한 변수

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.
    >> Update
        __init__
            isinstance : 첫 번째 파라미터값의 타입과 두 번째 파라미터값의 동등 여부를 확인하는 함수
        cleargrad
            메모리 절약을 위해서 한 번 할당했던 변수를 또 다시 사용해야 할 경우 기존 메모리에 있던 Variable.grad 정보가 남아있기 때문에 할당을 해제해주어야 제대로 동작을 수행하게 된다.
        backward
            반복문을 이용한 역전파 코드
            while loop 내에서 zip 함수를 활용하여 x.grad에 중복되게 값이 들

In [36]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [37]:
x = Variable(np.array(3.))
y = add(x, x)
y.backward()
print(x.grad)
x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


In [38]:
# weakref 모듈 임포트
# 현재 순전파 및 역전파 코드는 서로가 서로를 참조하는 순환 참조 형태의 코드이기 때문에
# weakref 모듈을 사용해서 메모리 참조 상에서 누수 문제를 방지
import weakref

class Function:
    """ Update Version (Level 2-3).
    Class Function : Variable 객체가 수행하는 함수들을 가지고 있는 클래스

    Parameter
    ---------
    input : Variable (Class instance)
    output : Variable (Class instance)
        input.data에 들어오는 ndarray 값을 함수 클래스를 통해 순전파된 결과인 y을 가지고 as_array 함수로 ndarray로 형변환하고 Variable의 __init__ 메서드로 값을 넣어 생성된 Variable 객체

    Functions
    ---------
    __call__
        위 클래스는 function 객체를 다루고 function 객체 내부에 input을 함수를 사용하듯이 값을 받음을 알 수 있음        
        inputs 파라미터를 리스트 데이터로 받아서 처리. 각각의 값은 Variable 객체로 구성되어 있음
        list comprehension을 사용하여 묶여진 객체들을 다시 인덱싱하여 결론을 도출함
        Level 2 버전의 Function 클래스와 큰 차이는 없으나 처음에 받는 인자를 가변인자를 받을 수 있도록 설정되어 있음
        generation 설정 : 이전까지의 Function 클래스의 경우 단일 노드들간의 연산이었으나, 지금부터는 여러 노드들에 대한 연산(ex. 더하기, 빼기 등)도 수행해야 하기 때문에 여러 노드들의 역전파를 효율적으로 수행하기 위해 함수 단계마다 세대(generation)를 설정하여 우선순위를 매긴 다음 역전파를 수행하도록 함
        return : outputs
        inputs 값을 가변인자로 받기 때문에 1개인 경우에 대한 예외처리가 필요하여 삼항 연산자를 활용해 조건에 따라 return되는 값을 다르게 하였음
        >> Update
            Config.enable_backprop : Config 라는 클래스를 생성하되 Flag 역할을 수행하도록 하여 해당 변수가 True값을 지니고 있을 경우 역전파 수행이 가능하도록 하였음. 이로 인해 생기는 효과는 역전파 수행이 필요 없을 경우 메모리를 절약할 수 있다는 장점이 있음
            weakref.ref : 현재 코드에서의 문제점은 메모리 누수가 발생할 수 밖에 없는 구조라는 것인데, 메모리 누수의 원인은 함수와 노드 사이의 연결이 서로가 서로를 참조하는 순환참조의 형태를 지니고 있기 때문임. 하여 weakref 모듈을 import하여 weakref가 참조하고 있는 변수의 메모리가 해제될 경우 이 변수도 정상적으로 메모리가 해제될 수 있도록 하였음
    forward, backward
        순전파 및 역전파 수행하는 클래스로써 기반클래스의 기본 포함 메서드. 파생 클래스에서 위 메서드들을 생성하지 않고 호출하게 되면 NotImplementedError를 raise하게 됨
        NotImplementedError : 파생 클래스에서 구현되지 않았음을 알리는 에러
    """
    def __call__(self, *inputs): # 1고지와 다르게 파라미터 값을 가변 인자로 받음
        inputs = [as_variable(x) for x in inputs] # 그에 따라서 inputs 값을 list comprehension을 사용하여 변환
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): # isinstance 함수를 사용하여 값의 타입 체크
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: # Config 클래스의 클래스 변수를 바로 체크
            self.generation = max([x.generation for x in inputs]) # 세대 체크 및 세대값 대입
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs] # weakref.ref를 사용해 객체 내부에 output값 넣기
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [39]:
def as_variable(obj): # 타입 체크
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [40]:
class Config:
    """
    Flag 설정 class
    특징
        Flag의 역할을 수행하는 클래스이기 때문에 클래스의 인스턴스를 생성할 필요 없이 클래스 변수를 바로 사용함
        메모리 상에서는 이 함수의 클래스 변수가 선언될 때만 메모리가 할당되었다가 메모리 해제 시점이 올 경우 자동 해제되므로 전역변수로 설정하는 것에 비해서 메모리의 효율을 높일 수 있다.
    """
    enable_backprop = True

In [41]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [42]:
# 가변인자를 parameter로 받아서 계산
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

f = Add()

y = f(x0, x1)
print(y.data)

5


In [43]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [44]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

y = add(x0, x1)
print(y.data)

5


In [45]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, *xs):
        if isinstance(xs, tuple):
            xs = xs[0]
        forward_list = [x.data for x in xs]
        if len(forward_list) > 1:
            y = forward_list[0] + forward_list[1]
        else:
            raise TypeError(f"'Length : {len(forward_list)}' isn't worked in Function (That's not correct length).")
        return (y,)

    def backward(self, gy):
        return gy, gy

In [46]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
# xs = [Variable(np.array(2))] # 오류 테스트
f = Add()
ys = f(xs)
# y = ys[0]
print(ys.data)

5


In [47]:
class Variable:
    """ Update Version (Level 2-2).
    Class Variable : np.array 값을 다루되 다른 멤버변수를 추가적인 특징으로 가져 다양한 정보를 모두 포함시키는 클래스.

    Parameter
    ---------
    data : np.array
        실제 연산에서 사용되는 값으로, 다양한 연산 및 정보 전달에도 사용
    grad : int, float
        역전파 수행 시, 현재 역전파 층에서의 gradient 값을 저장하여 다음 역전파에 전달하는 기능을 수행
    creator : callable function
        Variable 객체(인스턴스)를 연산하기 위한 함수가 무엇인지 저장하여 현재 객체는 creator에 들어온 함수값을 수행하기 위해 존재하는 것으로 판단하면 됨
    generation : Variable 객체가 어느 세대(generation)에 포함되어 있는지를 표현하기 위한 변수

    Functions
    ---------
    # 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    # 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.
    __init__
        isinstance : 첫 번째 파라미터값의 타입과 두 번째 파라미터값의 동등 여부를 확인하는 함수
        >> Update
            retain_grad : gradient 값(y().grad)을 유지시킬지 설정하는 변수. default = False
            name : 차후 Variable에 이름을 달아주기 위해 설정
    cleargrad
        메모리 절약을 위해서 한 번 할당했던 변수를 또 다시 사용해야 할 경우 기존 메모리에 있던 Variable.grad 정보가 남아있기 때문에 할당을 해제해주어야 제대로 동작을 수행하게 된다.
    backward
        반복문을 이용한 역전파 코드
        while loop 내에서 zip 함수를 활용하여 x.grad에 중복되게 값이 들어갈 경우 기존 노드에 더해지도록 만들어 x + x = 2x 라는 식을 예로 들었을 때, gradient 값이 2로 정상 출력되게 하였음
        >> Update
            add_func
                역전파 시 함수가 pop되면서 새로운 값이 들어올 때, 같은 메모리를 참조하는 경우에 중복 방지를 위해서 추가된 함수
                이미지 결과를 보면 더 이해가 쉽기 때문에 아래 코드블럭 중 Check로 Markdown 표시를 한 곳에서 이미지를 확인하기를 추천!!!
            weakref
                순환 참조로 인해 생기는 메모리 누수 문제를 해결하기 위해 import된 weakref 객체에 대해 실제로 값을 출력할 때는 Variable처럼 weakref도 어떠한 기본 데이터타입을 감싸고 있는 형태이므로 기본 생성자를 통해 객체를 호출함과 동시에 output을 사용해 실제 결과값을 확인함
    >> Update
        shape, ndim, size, dtype : numpy에 기본적으로 내장되어있는 메서드를 @property 데코레이터를 활용하여 바로 호출할 수 있도록 설정
        __len__ : data의 길이 반환
        __repr__ : print로 객체를 표현할 때 return할 값을 설정
        __mul__ : 다른 객체 또는 데이터타입과의 multiply 기능 지원
    """
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    # decorator를 활용하여 Variable 클래스의 data 객체는 numpy 타입이므로 해당 타입의 기능들을 Variable 객체의 기능처럼 가져옴 (shape, ndim, size, dtype).
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self): # ndarray의 길이 측정용 매직메서드
        return len(self.data) 

    def __repr__(self): # print문으로 객체 호출 시 반환할 값을 정하는 매직메서드
        if self.data is None:
            return "Variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p}) from __repr__' # print 함수로 Variable 호출 시 Variable.data의 값을 리턴하도록 설정

    def __mul__(self, other): # 곱셈기능 지원
        return mul(self, other)

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f): # 
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

In [48]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [49]:
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

In [50]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [51]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

In [52]:
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [53]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(y.grad)

None


In [54]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad() # 메모리 절약을 위해 변수를 재사용하는 경우 grad값이 남아있는 상태이므로 초기화 수행
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


In [ ]:
# from IPython.display import Image

# Image("Log.png")

## Check
a = square(x) 구문에서 a 값이 같은 메모리로 y 안에서 연산되므로 add_func 처리를 하지 않게 되면 a 값에 대해 func이 한 번 더 처리되게 되어 값이 64.0이 아닌 128.0이 나오게 됨    

![Check](Log.png)

In [55]:
x = Variable(np.array(2.0))         
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [56]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [57]:
# 메모리 할당 확인을 위한 모듈 설치

# !pip install memory_profiler
# %load_ext memory_profiler
# %memit

In [58]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()
print(x0.grad, x1.grad)
# %memit # peak memory: 138.91 MiB, increment: 0.00 MiB

2.0 1.0


In [59]:
# Config.enable_backprop = False
# x = Variable2(np.ones((100, 100, 100)))
# y = square(square(square(x)))
# # y.backward() # False일 경우 backward 사용 불가

In [60]:
# 데코레이터를 활용해서 후처리를 자동으로 수행하기 위한 함수 생성
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    print(f"*** Mode = {value}")
    old_value = getattr(Config, name) # getattr 옵션으로 기존 Config 클래스의 값을 저장
    setattr(Config, name, value) # setattr 옵션으로 Config 클래스 값 변경
    try:
        yield # 들여쓰기 내에 있는 본문을 실행하는 부분
    finally:
        setattr(Config, name, old_value) # setattr 옵션을 기존 옵션으로 다시 변경
        print("*** Done")

In [61]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
    print("y:", y.data)
    # %memit # peak memory: 100.75 MiB, increment: 0.00 MiB

*** Mode = False
y: 4.0
*** Done


In [62]:
def no_grad(): # using_config 옵션 설정은 스위치 버튼이나 사용 시마다 쳐야하는 불편함을 제거하고자 추가적으로 함수 생성
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)
    # %memit # peak memory: 100.71 MiB, increment: 0.00 MiB

*** Mode = False
*** Done


#### contextlib 예제

In [63]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
    finally:
        print('done')

with config_test():
    print('process...')

start
process...
done


#### Mul Class

In [64]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy): # 곱셈 연산에서의 역전파 시, 값이 서로 교차되어 들어가는 부분이 반영되어있음
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1*gy, x0*gy

In [65]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [66]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))
c = Variable(np.array(1.))

y = add(mul(a, b), c)
y.backward()

In [67]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [68]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))

y = a * b
print(y)

variable(6.0) from __repr__


In [69]:
# Variable 내에 매직 메서드를 추가하는 방식을 클래스 내에 매직메서드를 선언하는 방식을 사용하지 않고
# 기존에 차용했던 mul, add 함수를 그대로 대입하여 지정이 가능함
Variable.__mul__ = mul
Variable.__add__ = add

# add 및 mul의 경우 연산자 기준으로 좌측은 클래스 객체를, 우측으로는 다른 타입의 값을 받아들이는데
# radd 및 rmul의 경우에는 연산자 기준으로 좌측에 다른 타입의 값이 존재하고 우측에 클래스 객체가 있을 때의
# 연산자 처리를 하기 위해 만들어진 매직메서드로써 파이썬 클래스 내부에 기본적으로 내장되어있는 기능임
# 값이 들어갈 때, 인자 값을 나열된 순서대로 넣지 않고 뒤 순서의 값을 앞으로 넣어
# 앞에 존재하는 Variable 객체의 메서드를 호출하도록 되어 있음
Variable.__rmul__ = mul
Variable.__radd__ = add

In [70]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))
c = Variable(np.array(1.))

y = a * b + c
y.backward()

In [71]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [72]:
x = Variable(np.array(2.))
y = x + np.array(3.0) # 이 경우 Variable 객체의 __add__ 매직메서드가 실행됨
print(y)

variable(5.0) from __repr__


In [73]:
x = Variable(np.array(2.))
y = x + 3.0
print(y)

variable(5.0) from __repr__


In [74]:
# 해당 식의 경우 x 앞에 2.0이라는 float 값이 곱해지는 형태로 곱셈연산의 경우 앞뒤가 뒤바뀌어서 들어가도 무방하므로 __rmul__ 메서드가 호출되고
# 2차적으로는 곱셈연산이 되었던 값이 rmul에 의해 Variable 객체의 값을 지닌 상태로 add가 정상 수행되므로 __add__ 메서드가 호출되어 결과값이 나오게 됨
y = 2.0 * x + 1.0 
print(y)

variable(5.0) from __repr__


#### 기타 함수 및 메서드 추가

In [75]:
# Negative (-1 곱하기)
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

In [76]:
def neg(x):
    return Neg()(x)

In [77]:
Variable.__neg__ = neg

In [78]:
# Subtraction
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

In [79]:
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

In [80]:
Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [81]:
# Division
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [82]:
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

In [83]:
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [84]:
# Power
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

In [85]:
def pow(x, c):
    return Pow(c)(x)

In [86]:
Variable.__pow__ = pow

In [87]:
x = Variable(np.array(2.0))
y = x ** 0
print(y)

variable(1.0) from __repr__
